In [1854]:
import torch
from torch import nn,optim
import torch.nn.functional
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os as os
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,multilabel_confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from scipy.stats import kurtosis


from sklearn.model_selection import cross_val_score

#import warnings

#warnings.simplefilter(action='ignore', category=FutureWarning)



<h1>Train data prep</h1>

There is an observation every second. <br>
There are 1200 observation for every dataframe.<br>
Columns P1-P7 are pressures at the measurement point.<br>

In [1855]:
path_train = '/Users/alexdediu/git-projects/Data-Challenge-PHM-Asia-Pacific-2023/dataset/train'
path_data = "/data"
path_labels = "/labels.xlsx"


dfs_data = []

for filename in os.listdir(path_train+path_data):
    if filename.endswith('.csv'):
       
        percorso_file = os.path.join(path_train+path_data, filename)
        df = pd.read_csv(percorso_file)
        dfs_data.append(df)

df_data = pd.concat(dfs_data, ignore_index=True)

df_data



,TIME,P1,P2,P3,P4,P5,P6,P7
0,0.000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
1,0.001,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
2,0.002,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
3,0.003,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
4,0.004,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
...,...,...,...,...,...,...,...,...
212572,1.196,1.952063,1.908461,1.934484,1.837319,1.921240,1.912475,1.919641
212573,1.197,1.949134,1.915494,1.885116,1.827055,1.916485,1.907144,1.881396
212574,1.198,1.956077,1.922930,1.902461,1.887422,1.925576,1.916930,1.927116
212575,1.199,1.968020,1.958794,1.962051,1.912295,1.942300,1.941114,1.904265


In [1856]:
autocorr_dfs = []


for df in dfs_data:
    autocorr_data = df.apply(lambda x: x.autocorr(), axis=0)  
    autocorr_data = autocorr_data.rename(lambda x: x + '_autocorr') 
    autocorr_dfs.append(autocorr_data)

df_autocorr = pd.concat(autocorr_dfs, axis=1).T


df_autocorr = df_autocorr.drop("TIME_autocorr",axis=1)

df_autocorr

,P1_autocorr,P2_autocorr,P3_autocorr,P4_autocorr,P5_autocorr,P6_autocorr,P7_autocorr
0,0.838199,0.835964,0.681513,0.768680,0.845926,0.731044,0.786788
1,0.829271,0.844887,0.683057,0.630370,0.838946,0.497369,0.750026
2,0.821104,0.883735,0.850891,0.869841,0.844558,0.886028,0.883785
3,0.821021,0.882963,0.845620,0.865571,0.843003,0.891434,0.883220
4,0.820228,0.884896,0.854957,0.873245,0.845703,0.894475,0.887836
...,...,...,...,...,...,...,...
172,0.820718,0.884506,0.851112,0.870047,0.845018,0.891609,0.868503
173,0.820703,0.885203,0.855027,0.872897,0.845905,0.886914,0.885235
174,0.820940,0.883903,0.853221,0.871915,0.843533,0.890856,0.882327
175,0.813160,0.830937,0.664702,0.701021,0.857067,0.700038,0.871910


In [1857]:
condition_map = {'Normal':1,'Fault':2,'Anomaly':3}
bp_map = {'No':0,'Yes':1}
df_labels = pd.read_excel(path_train+path_labels,header=1)

df_labels = df_labels.rename(columns={"Unnamed: 0":"Case","Unnamed: 1":"Spacecraft","Unnamed: 2":"Condition"})

df_labels["Condition"] = df_labels["Condition"].map(condition_map)
bp_columns = [col for col in df_labels.columns if ("BP" in col) or("BV" in col)]
print(bp_columns)

for col in bp_columns:
    df_labels[col] = df_labels[col].map(bp_map)

df_labels



['BP1', 'BP2', 'BP3', 'BP4', 'BP5', 'BP6', 'BP7', 'BV1']


,Case,Spacecraft,Condition,SV1,SV2,SV3,SV4,BP1,BP2,BP3,BP4,BP5,BP6,BP7,BV1
0,1,1,1,100,100,100,100,0,0,0,0,0,0,0,0
1,2,1,1,100,100,100,100,0,0,0,0,0,0,0,0
2,3,1,1,100,100,100,100,0,0,0,0,0,0,0,0
3,4,1,1,100,100,100,100,0,0,0,0,0,0,0,0
4,5,1,1,100,100,100,100,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,173,3,3,100,100,100,100,0,0,0,1,0,0,0,0
173,174,3,3,100,100,100,100,0,0,0,0,1,0,0,0
174,175,3,3,100,100,100,100,0,0,0,0,0,1,0,0
175,176,3,3,100,100,100,100,0,0,0,0,0,0,1,0


In [1858]:
dfs_features = []
for d in dfs_data:
    d = d.drop("TIME", axis=1)
    mean = d.mean()
    std = d.std()
    skewness = d.skew()
    kurt = d.kurtosis()
    min = d.min()
    max= d.max()
    percentile_25 = d.quantile(0.25)  
    percentile_50 = d.quantile(0.50) 
    percentile_75 = d.quantile(0.75) 

    #print(type(mean))
    df_stats = pd.DataFrame()
    for col in d.columns:
        df_stats.loc[0, col + "_temp_mean"] = mean[col]
        df_stats.loc[0, col + "_temp_std"] = std[col]
        df_stats.loc[0, col + "_temp_skewness"] = skewness[col]
        df_stats.loc[0, col + "_temp_kurtosis"] = kurt[col]
        #df_stats.loc[0, col + "_temp_25th_percentile"] = percentile_25[col]
        df_stats.loc[0, col + "_temp_50th_percentile"] = percentile_50[col]
        #df_stats.loc[0, col + "_temp_75th_percentile"] = percentile_75[col]
        df_stats.loc[0, col + "_temp_min"] = min[col]
        df_stats.loc[0, col + "_temp_max"] = max[col]
    #print(df_stats)
    dfs_features.append(df_stats)

df_temp_desc = pd.concat(dfs_features, ignore_index=True)


df_temp_desc

,P1_temp_mean,P1_temp_std,P1_temp_skewness,P1_temp_kurtosis,P1_temp_50th_percentile,P1_temp_min,P1_temp_max,P2_temp_mean,P2_temp_std,P2_temp_skewness,...,P6_temp_50th_percentile,P6_temp_min,P6_temp_max,P7_temp_mean,P7_temp_std,P7_temp_skewness,P7_temp_kurtosis,P7_temp_50th_percentile,P7_temp_min,P7_temp_max
0,1.984256,0.289766,1.851666,18.529528,1.956981,0.457024,4.120726,1.976893,0.456648,1.908179,...,1.917763,-0.022742,5.226310,1.971555,0.598999,0.926702,6.520890,1.917877,0.008186,5.040325
1,1.985080,0.291307,1.802978,18.402516,1.956283,0.448800,4.115485,1.977530,0.453900,1.761304,...,1.886752,-0.005303,5.651999,1.966778,0.609958,0.898882,6.927636,1.932216,-0.000112,5.044157
2,1.984897,0.308460,1.638402,18.402486,1.958770,0.436712,4.254711,1.977711,0.446403,1.077640,...,1.906730,-0.003679,5.030109,1.971533,0.567295,1.038752,7.334376,1.908164,-0.002230,5.010291
3,1.984909,0.308212,1.619896,18.322574,1.958090,0.437263,4.265872,1.977935,0.446723,1.082381,...,1.918237,-0.001637,5.034114,1.973697,0.576946,1.041171,6.930139,1.914307,-0.003876,5.006885
4,1.985043,0.308653,1.701832,18.771234,1.959761,0.436888,4.311454,1.977717,0.447126,1.128660,...,1.909371,-0.002783,4.997056,1.968631,0.574143,0.962339,6.688594,1.910967,-0.002193,4.995528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,1.984999,0.308539,1.682806,18.656679,1.960032,0.436793,4.295627,1.977797,0.447068,1.125532,...,1.912444,-0.002923,5.015434,1.974020,0.594715,1.000985,6.151939,1.913541,-0.007141,5.000936
173,1.984931,0.308624,1.669368,18.490675,1.957459,0.436435,4.256737,1.977437,0.446069,1.084697,...,1.909310,-0.002919,5.014037,1.972029,0.579382,0.958695,6.562989,1.909395,-0.003786,4.999330
174,1.984892,0.308434,1.624987,18.275383,1.957271,0.436619,4.232319,1.977673,0.446080,1.056653,...,1.920334,-0.002700,5.032940,1.971729,0.566238,1.046536,7.356719,1.913895,-0.002312,5.008263
175,1.985246,0.271765,1.993451,21.082582,1.976075,0.579673,4.103252,1.976562,0.428722,2.042569,...,1.924745,0.009534,5.748213,1.976230,0.589475,1.156362,5.892525,1.915425,0.266158,4.747359


In [1859]:
df_y = df_labels
df_y = df_y.drop(["Case","Spacecraft"],axis=1)
df_y

,Condition,SV1,SV2,SV3,SV4,BP1,BP2,BP3,BP4,BP5,BP6,BP7,BV1
0,1,100,100,100,100,0,0,0,0,0,0,0,0
1,1,100,100,100,100,0,0,0,0,0,0,0,0
2,1,100,100,100,100,0,0,0,0,0,0,0,0
3,1,100,100,100,100,0,0,0,0,0,0,0,0
4,1,100,100,100,100,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,3,100,100,100,100,0,0,0,1,0,0,0,0
173,3,100,100,100,100,0,0,0,0,1,0,0,0
174,3,100,100,100,100,0,0,0,0,0,1,0,0
175,3,100,100,100,100,0,0,0,0,0,0,1,0


In [1860]:
fs = 1000  # frequenza di campionamento (1 kHz)
dt = 1 / fs  # intervallo di campionamento

<h2>Analisi in frequenza</h2>

In [1861]:
dfs_freq = []
#dfs_amplitude = []
dfs_fft = []
dfs_spectral_features = []

def calculate_fft_features(df):


    for _, df_data in enumerate(df):
        df_freq = pd.DataFrame()
        #df_amplitude = pd.DataFrame()
        df_fft = pd.DataFrame()
        df_spectral_features = pd.DataFrame()

        for col in df_data.columns:
            if col != "TIME":
                df_data[col] = df_data[col] - df_data[col].mean()

                fft_result = np.fft.fft(df_data[col]).real
         
                freq = np.fft.fftfreq(len(fft_result), d=dt)
                amplitude = np.abs(fft_result)

                index_of_max_amplitude = np.argmax(amplitude)

                frequency_dominate = freq[index_of_max_amplitude]

                df_dominant_freq = pd.DataFrame({col + '_dominant_freq': [np.abs(frequency_dominate)]})

                df_freq = pd.concat([df_freq, df_dominant_freq], axis=1)

                df_fft[col + "_fft"] = fft_result
                #df_amplitude[col + '_amplitude'] = amplitude

                #impulse_factor = np.abs(fft_result[index_of_max_amplitude]) / np.sum(amplitude)
                amplitude_nonzero = np.where(amplitude == 0, 1e-9, amplitude)
                #entropy = -np.sum((amplitude_nonzero / np.sum(amplitude_nonzero)) * np.log2(amplitude_nonzero / np.sum(amplitude_nonzero)))

                energy = np.sum(amplitude ** 2)
                kurt = kurtosis(amplitude, nan_policy='omit')

                df_spectral_features.loc[0, col + '_kurtosis'] = kurt
                df_spectral_features.loc[0, col + '_energy'] = round(energy)

        dfs_fft.append(df_fft)
        dfs_freq.append(df_freq)
        #dfs_amplitude.append(df_amplitude)
        dfs_spectral_features.append(df_spectral_features)

    df_fft_complete = pd.concat(dfs_fft, axis=0)
    df_freq_complete = pd.concat(dfs_freq, axis=0)
    #df_amplitude_complete = pd.concat(dfs_amplitude, axis=0)
    df_spectral_features_complete = pd.concat(dfs_spectral_features, axis=0)

    return df_freq_complete, df_fft_complete, df_spectral_features_complete


def separate_energy_kurtosis(df_spectral_features_complete):
    df_energy = df_spectral_features_complete.filter(like='_energy')
    df_kurtosis = df_spectral_features_complete.filter(like='_kurtosis')
    return df_energy, df_kurtosis


df_freq, df_fft, df_spectral_features = calculate_fft_features(dfs_data)
df_energy, df_kurtosis = separate_energy_kurtosis(df_spectral_features)


In [1862]:
def reset_index_and_drop(df):
    df_reset_index = df.reset_index()
    df_dom = df_reset_index.drop("index", axis=1)
    return df_dom

# Esempio di utilizzo della funzione per ciascun DataFrame
df_freq = reset_index_and_drop(df_freq)
df_fft = reset_index_and_drop(df_fft)
#df_amplitude_dom = reset_index_and_drop(df_amplitude)
df_energy = reset_index_and_drop(df_energy)
df_kurtosis = reset_index_and_drop(df_kurtosis)

In [1863]:
import pandas as pd

dfs_means = []

for df in dfs_fft:
    df_mean = pd.DataFrame()

    for col in df.columns:
        col_name_mean = col + "_freq_mean"
        col_name_std = col + "_freq_std"
        col_min = col +"_freq_min"
        col_max = col +"_freq_max"

        df_mean[col_name_mean] = [df[col].mean()]
        df_mean[col_name_std] = [df[col].std()]
        df_mean[col_min] = [df[col].min()]
        df_mean[col_max] = [df[col].max()]

        
    dfs_means.append(df_mean)

df_freq_means = pd.concat(dfs_means, axis=0)
df_freq_means=df_freq_means.reset_index()


df_freq_means


,index,P1_fft_freq_mean,P1_fft_freq_std,P1_fft_freq_min,P1_fft_freq_max,P2_fft_freq_mean,P2_fft_freq_std,P2_fft_freq_min,P2_fft_freq_max,P3_fft_freq_mean,...,P5_fft_freq_min,P5_fft_freq_max,P6_fft_freq_mean,P6_fft_freq_std,P6_fft_freq_min,P6_fft_freq_max,P7_fft_freq_mean,P7_fft_freq_std,P7_fft_freq_min,P7_fft_freq_max
0,0,0.015744,6.904793,-68.361033,61.105875,0.023107,10.878241,-108.467318,98.948128,0.026702,...,-111.454509,107.248343,0.029896,15.296213,-127.290020,128.326413,0.028445,14.568567,-130.622168,116.019118
1,0,0.014920,6.880439,-68.494269,57.468080,0.022470,10.748185,-109.633039,92.045014,0.026479,...,-112.952567,98.508436,0.037497,16.780998,-130.469907,123.383653,0.033222,14.850785,-125.862193,101.308178
2,0,0.015103,7.498994,-77.785882,70.922751,0.022289,10.940459,-119.067000,112.736024,0.027594,...,-126.379154,125.186435,0.039955,14.065630,-142.130339,147.440702,0.028467,13.819352,-144.754078,147.990436
3,0,0.015091,7.479236,-77.414740,70.119641,0.022065,10.922330,-118.703417,111.517628,0.027757,...,-125.372773,123.503610,0.042785,14.147088,-141.572891,144.516607,0.026303,14.297431,-147.093982,152.813581
4,0,0.014957,7.500266,-77.603568,70.669438,0.022283,10.955229,-119.164606,112.478537,0.027649,...,-124.839640,123.862606,0.038543,14.061584,-142.206363,146.654276,0.031369,14.145160,-144.219433,154.143091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,0,0.015001,7.499226,-77.645974,70.771997,0.022203,10.954867,-119.178651,112.556451,0.027819,...,-125.027273,123.908423,0.036455,14.110897,-142.145523,146.791795,0.025980,14.901947,-152.052807,146.310563
173,0,0.015069,7.508428,-77.900253,71.237182,0.022563,10.944235,-119.213102,113.228982,0.027357,...,-126.108566,125.748368,0.038378,14.054185,-141.598810,147.578174,0.027971,14.371093,-146.768987,157.578101
174,0,0.015108,7.498982,-77.816951,70.981464,0.022327,10.934569,-119.039114,112.852860,0.027488,...,-127.093925,125.048277,0.029118,14.350468,-142.595105,147.276263,0.028271,13.838693,-144.479081,149.453246
175,0,0.014754,6.345879,-42.069359,46.662874,0.023438,9.937882,-73.037716,70.598274,0.024893,...,-77.547061,82.657275,0.025566,14.207705,-82.637945,99.929768,0.023770,13.084297,-102.224735,109.796958


<h4>ML Models</h4>

In [1864]:
df_x=[]
for df in dfs_freq:
    df_x.append(df.iloc[0].values.tolist())

In [1865]:
df_x = pd.concat([df_temp_desc,df_autocorr,df_freq,df_energy],axis=1)
#df_x=df_x.drop("index",axis=1)
df_x['Spacecraft'] = df_labels['Spacecraft']


In [1866]:
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y["Condition"], test_size=0.33, random_state=42)

X_train


,P1_temp_mean,P1_temp_std,P1_temp_skewness,P1_temp_kurtosis,P1_temp_50th_percentile,P1_temp_min,P1_temp_max,P2_temp_mean,P2_temp_std,P2_temp_skewness,...,P6_dominant_freq,P7_dominant_freq,P1_energy,P2_energy,P3_energy,P4_energy,P5_energy,P6_energy,P7_energy,Spacecraft
22,1.984923,0.308383,1.629303,18.332697,1.958684,0.437003,4.260626,1.977697,0.446361,1.070346,...,64.945878,64.945878,67266.0,143152.0,185784.0,207898.0,162908.0,237923.0,233689.0,1
145,1.983783,0.276185,1.853412,20.703634,1.967847,0.522370,4.027160,1.978689,0.423001,1.545888,...,59.950042,59.950042,49578.0,120053.0,205067.0,227768.0,126493.0,222014.0,223868.0,3
97,1.985147,0.301867,1.693425,18.233081,1.955910,0.442322,4.215108,1.980114,0.455083,1.576530,...,59.950042,67.443797,62240.0,143728.0,218999.0,234139.0,147509.0,220715.0,248946.0,3
69,1.985042,0.292627,1.814652,18.667090,1.955712,0.442304,4.142868,1.977969,0.454503,1.744788,...,59.950042,59.950042,57444.0,139127.0,202077.0,268371.0,138302.0,322775.0,275862.0,2
166,1.984888,0.292399,1.834561,18.765211,1.955984,0.440147,4.161710,1.977682,0.453629,1.816406,...,59.950042,59.950042,57422.0,138454.0,202251.0,241886.0,140610.0,254836.0,251214.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,1.984799,0.301243,1.674440,18.134582,1.955870,0.442684,4.184923,1.978020,0.452740,1.537143,...,59.950042,67.443797,61886.0,141972.0,230184.0,240185.0,148852.0,220811.0,213191.0,3
106,1.984939,0.307453,1.614792,18.055510,1.957590,0.448898,4.187418,1.976414,0.447825,0.960542,...,64.945878,64.945878,67266.0,145008.0,185773.0,202090.0,163310.0,242025.0,234384.0,1
14,1.984879,0.308560,1.645548,18.360354,1.957165,0.436263,4.227153,1.977560,0.446022,1.066640,...,64.945878,64.945878,67718.0,143921.0,187167.0,208855.0,164562.0,244229.0,257442.0,1
92,1.991117,0.233919,1.434403,13.906710,1.963140,0.919984,3.600334,1.984435,0.382075,1.146688,...,64.945878,67.443797,39341.0,111026.0,198504.0,226208.0,100736.0,201098.0,175750.0,3


In [1867]:
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [1868]:
clf = GradientBoostingClassifier(n_estimators=200, random_state=42)

# Addestra il classificatore
clf.fit(X_train, y_train)

# Effettua le previsioni sul set di test
y_pred = clf.predict(X_test)

# Valuta l'accuratezza del modello
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6271186440677966


In [1869]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.64
Precision: 0.6485875706214689
Recall: 0.6440677966101694
F1 Score: 0.6280107047279214
